<a href="https://colab.research.google.com/github/SUDHARSAN270/Machine_learning/blob/main/audio_construction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import torchaudio
import torchaudio.transforms as transforms

In [4]:
train_dataset = torchaudio.datasets.LIBRISPEECH(
    root='/media',
    url='train-clean-100',
    download=True
)

100%|██████████| 5.95G/5.95G [05:21<00:00, 19.9MB/s]


In [11]:
# Define a transform to apply to the audio data
transform = transforms.MelSpectrogram(
    sample_rate=16000, n_mels=128
)



# Load the LibriSpeech dataset
original_dataset = torchaudio.datasets.LIBRISPEECH(
    root='/media',
    url='train-clean-100',
    download=True
)


# Define a custom dataset class
class CustomAudioDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transform):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        waveform, sample_rate, *additional_data = self.dataset[idx]
        if self.transform:
            waveform = self.transform(waveform)
        return waveform, additional_data

# Instantiate the custom dataset with the transform
train_dataset = CustomAudioDataset(original_dataset, transform=transform)

# Function to pad sequences to the same length
def pad_sequence(batch):
    max_length = max([item.size(-1) for item in batch])
    padded_batch = []
    for item in batch:
        padding = max_length - item.size(-1)
        if padding > 0:
            padded_item = torch.nn.functional.pad(item, (0, padding))
        else:
            padded_item = item
        padded_batch.append(padded_item)
    padded_batch = torch.stack(padded_batch)
    return padded_batch

# Custom collate function to handle variable-length audio
def collate_fn(batch):
    tensors, targets = zip(*batch)
    tensors = [item.unsqueeze(0) for item in tensors]  # Add channel dimension
    tensors = pad_sequence(tensors)  # Pad sequences to the same length
    return tensors, targets

# Define a DataLoader with the custom collate function
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

# Iterate through the DataLoader
for batch_idx, (data, target) in enumerate(train_loader):
    print(data.shape, target)
    if batch_idx == 0:
        break






torch.Size([32, 1, 1, 128, 1353]) (['HER HEAD WAS A LITTLE ON ONE SIDE IN THE ATTITUDE OF ONE WHO LISTENS INTENTLY SLOWLY AND CAREFULLY MERCER THOUGHT OUT THE WHOLE STORY OF HIS EXPERIENCES WITH THE GIRL UNTIL SHE HAD PLUNGED INTO THE POOL THEN I SAW AGAIN THE BEACH', 8088, 284756, 94], ['HE WILL ALWAYS BE BLIND DEAR BUT LIFE HOLDS OTHER THINGS BESIDE SIGHT WE MUST NEVER SAY NO HOPE WILL HE LIVE THERE IS NO REASON HE SHOULD NOT LIVE', 226, 122538, 33], ["ABOUT FIVE HUNDRED MILES FROM HEART'S CONTENT", 839, 130898, 43], ['WAS SUFFICIENT TO ATTRACT THE MOSQUITO FROM EVERY PART OF THE COUNTRY FOR SOME TIME THE GARRISON ENDEAVORED TO HOLD OUT BUT IT WAS ALL IN VAIN THE MOSQUITOS PENETRATED INTO EVERY CHINK AND CREVICE AND GAVE THEM NO REST DAY NOR NIGHT', 8051, 295385, 36], ['IS A VERY GREAT EVIL IT WARPS AND DEPLETES PUBLIC INFORMATION IT PREVENTS THE JUST CRITICISM OF PUBLIC SERVANTS ABOVE ALL IT GIVES IMMENSE AND IRRESPONSIBLE POWER TO A HANDFUL OF WEALTHY MEN', 7264, 92314, 20], ["PROV

In [12]:
class audio_constructor(nn.Module):
  def __init__(self,input_dim, hidden_dim, output_dim, num_layers, dropout = 0.2):
    super(audio_constructor, self).__init__()
    self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, dropout= dropout, batch_first = True)
    self.Dropout = nn.Dropout(dropout)
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self,x):
    h0 = torch.zeros(self.num_layers, x.size(), hidden_dim).to_device()
    c0 = torch.zeros(self.num_layers, x.size(), hidden_dim).to_device()
    out, (hn, cn) = self.lstm(x, (h0, c0))
    out =self.Dropout(out[:,-1,:])
    out = self.fc(out)
    return out


input_dim = 1028
hidden_dim = 1028
output_dim = 100
num_layers = 10
learning_rate = 0.001
dropout_rate = 0.2
num_epochs = 100


model = audio_constructor(input_dim, hidden_dim, output_dim, num_layers, dropout_rate)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

for epoch in range(num_epochs):
  for i, (inputs, labels) in enumerate(train_Loader):
    outputs = model(audio_tensor)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


NameError: name 'Ture' is not defined